In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

## Customer_Table

In [0]:
customer_schema =StructType([
    StructField("customer_id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("email", StringType(), True),
    StructField("phone", StringType(), True),
    StructField("address", StringType(), True),
    StructField("credit_score", IntegerType(), True),
    StructField("join_date", DateType(), True),
    StructField("last_update", TimestampType(), True)
])   

customer_df=spark.read.csv("dbfs:/FileStore/capstone/dataset/customers.csv", header=True, schema=customer_schema)
customer_df.write.format("delta").mode("append").saveAsTable("bronze.customers")


### Branch Table

In [0]:
branches_schema = StructType([
    StructField("branch_id", StringType(), True),
    StructField("branch_name", StringType(), True),
    StructField("location", StringType(), True),
    StructField("timezone", StringType(), True)
])

branch_df=spark.read.csv("dbfs:/FileStore/capstone/dataset/branches.csv", header=True, schema=branches_schema)
branch_df.write.format("delta").mode("append").saveAsTable("bronze.branches")
branch_df.display()

branch_id,branch_name,location,timezone
B0000,North Branch,Michaelshire,PST
B0001,East Branch,Trantown,GMT
B0002,Central Branch,Burtonburgh,EST
B0003,East Branch,Andersonmouth,EST
B0004,West Branch,Port Joseph,AEST
B0005,Downtown Branch,West Nathanland,GMT
B0006,West Branch,New Jameschester,GMT
B0007,West Branch,Catherinechester,PST
B0008,West Branch,Brownfort,GMT
B0009,North Branch,Port Henryshire,PST


### Streamming Data from transaction table

In [0]:

transactions_schema = StructType([
    StructField("transaction_id", StringType(), True),
    StructField("customer_id", IntegerType(), True),
    StructField("branch_id", IntegerType(), True),
    StructField("channel", StringType(), True),
    StructField("transaction_type", StringType(), True),
    StructField("amount", DoubleType(), True),
    StructField("currency", StringType(), True),
    StructField("timestamp", TimestampType(), True),
    StructField("status", StringType(), True)
])

def ingest():
  source_df = (spark.readStream
                      .format("cloudFiles")
                      .option("cloudFiles.format", "csv")  
                      .option("header", "true") 
                      .option("timestampFormat","yyyy-MM-dd")                  
                      .option("cloudFiles.schemaLocation", "dbfs:/FileStore/capstone/chekpoint/schema")
                      .option("cloudFiles.inferColumnTypes", "true")                    
                      .load("dbfs:/FileStore/capstone/transactions")
  )

  write_query = (source_df.writeStream
                          .format("delta")
                          .option("checkpointLocation", "dbfs:/FileStore/capstone/chekpoints/")
                          .option("mergeSchema", "true")
                          .outputMode("append")                         
                          .toTable("bronze.transactions")
  )

ingest()

org.apache.spark.sql.catalyst.analysis.NamespaceAlreadyExistsException: [SCHEMA_ALREADY_EXISTS] Cannot create schema `bronze` because it already exists.
Choose a different name, drop the existing schema, or add the IF NOT EXISTS clause to tolerate pre-existing schema. SQLSTATE: 42P06
	at org.apache.spark.sql.errors.QueryCompilationErrors$.namespaceAlreadyExistsError(QueryCompilationErrors.scala:1693)
	at org.apache.spark.sql.execution.datasources.v2.CreateNamespaceExec.run(CreateNamespaceExec.scala:53)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.$anonfun$result$2(V2CommandExec.scala:48)
	at org.apache.spark.sql.execution.SparkPlan.runCommandWithAetherOff(SparkPlan.scala:178)
	at org.apache.spark.sql.execution.SparkPlan.runCommandInAetherOrSpark(SparkPlan.scala:189)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.$anonfun$result$1(V2CommandExec.scala:48)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:94)
	at org.apache.spark.s